In [ ]:
from __future__ import print_function
import KerasTools as KT
import numpy as np
import time

from keras import backend as K
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

# Use a MobileNet as convolutional base for feature extraction
from keras.applications import MobileNet

K.clear_session()

# The example here uses the smallest possible mobilenet architecture, 
# but still achieves reasonable accuracy.
#
# Increase the alpha parameter and the image size 
# to get higher accuracy in the final network 
# for cost of longer extraction / training time
alpha = 0.25                 # MobileNet model width, one of [0.25, 0.5, 0.75, 1.0]
input_shape = (128, 128, 3)  # Image size, one of [128, 160, 192, 224]

conv_base = MobileNet(weights='imagenet',      # Use pre-trained ImageNet weights
                      include_top = False,     # No classifier, only the convolutional base: 
                      input_shape=input_shape, 
                      alpha=alpha,
                      pooling='avg')

# Needed here when called from a webapp like jupyter notebook; tensorflow does not play well with threads
conv_base._make_predict_function()

In [ ]:
conv_base.summary()

In [ ]:
def extract_features(conv_base, directory, sample_count):
    start_time = time.time()
    batch_size = 10
    features = np.zeros(shape=(sample_count, conv_base.output.shape[1]))
    labels = np.zeros(shape=(sample_count, 2))
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    generator = datagen.flow_from_directory(
              directory,
              target_size=conv_base.input_shape[1:3],
              batch_size=batch_size,
              class_mode='categorical')
    print(generator.class_indices)
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        KT.update_progress(directory, float(i*batch_size)/sample_count)
        if i * batch_size >= sample_count:
          # Note that since generators yield data indefinitely in a loop,
          # we must `break` after every image has been seen once.
            break
    end_time = time.time()
    print(str(end_time-start_time)+ 's')
    return features, labels

In [ ]:
train_dir = '../Datasets/Pet_Dataset/train'
test_dir  = '../Datasets/Pet_Dataset/test'

train_features, train_labels = extract_features(conv_base, train_dir, 4800)
test_features,  test_labels  = extract_features(conv_base, test_dir,  1200)

In [ ]:
# Build a custom DNN classifier
def build_classifier():
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu', input_dim=int(alpha*1024)))
    model.add(layers.Dense(2, activation='softmax'))
    model.compile(optimizer=optimizers.sgd(momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
    return model
    
classifier = build_classifier()
classifier.summary()

In [ ]:
# Train the classifier using the extracted features
batch_size = 200
history = classifier.fit(train_features, train_labels, 
                         batch_size=batch_size, epochs=200,
                         validation_split=0.20)

In [ ]:
KT.plot_history(history.history)

In [ ]:
# Train the final classifier at the onset of overfitting
final_epochs = 50

classifier = build_classifier()
classifier.fit(train_features, train_labels, 
               batch_size=batch_size, epochs=final_epochs)

test_loss, test_acc = classifier.evaluate(test_features, test_labels)
print()
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)
history.history['epochs'] = final_epochs
history.history['test_loss'] = test_loss
history.history['test_acc'] = test_acc

In [ ]:
KT.plot_history(history.history)

In [ ]:
# Generate final model with both convolutional base & classifier
model = models.Sequential()
model.add(conv_base)
model.add(classifier)
model.compile(optimizer=optimizers.sgd(momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

# Save model to disk
model.save('./cats_and_dogs_trained.h5')